# Final Project

In [7]:
import warnings
warnings.filterwarnings('ignore')

from helper import get_train_test, lowercase, tokenize, remove_punctuation, remove_stopwords, \
    remove_non_letters, stemming, correct_spelling, reduce_lengthening, subsampling, save_preprocessed_data, \
    get_categories, get_train_test_data, get_label_encoder
    
config = {
    'lowercase': True,
    'stemming': True,
    'remove_stopwords': True,
    'remove_non_letters': True,
    'remove_punctuation': True,
    'correct_spelling': True,
    'reduce_lengthening': True,
    'subsampling': False,
}

## Preprocessing

In [1]:
train, test = get_train_test_data('train_data.txt', 'test_data.txt')

# subsampling
if config['subsampling']:
    subsampling(train)
    subsampling(test)

# lowercasing
if config['lowercase']:
    lowercase(train)
    lowercase(test)

# removing non letters
if config['remove_non_letters']:
    remove_non_letters(train)
    remove_non_letters(test)

# removing punctuation
if config['remove_punctuation']:
    remove_punctuation(train)
    remove_punctuation(test)

# tokenizing
tokenize(train)
tokenize(test)

# reducing length
if config['reduce_lengthening']:
    reduce_lengthening(train)
    reduce_lengthening(test)
    
# correcting spelling
if config['correct_spelling']:
    correct_spelling(train)
    correct_spelling(test)

# removing stopwords
if config['remove_stopwords']:
    remove_stopwords(train)
    remove_stopwords(test)
    
# stemming
if config['stemming']:
    stemming(train)
    stemming(test)
    
# saving preprocessed data
save_preprocessed_data(train, 'train_data.pkl')
save_preprocessed_data(test, 'test_data.pkl')

## Checkpoint

In [8]:
from helper import load_preprocessed_data

train, test = load_preprocessed_data('train_data.pkl', 'test_data.pkl')

## Classifications

### Nearest Centroid

In [5]:
# TODO

### LSTM (Deep Learning)

In [9]:
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.preprocessing import text, sequence
from numpy import argmax

hyper_params = {
    'validation_split': 0.1,
    'batch_size': 32,
    'epochs': 10,
    'embedding_size': 128,
    'keep_probability': 0.9,
    'lstm_size': 50,
    'dense_size': 50,
    'max_sequence': 100
}

categories = get_categories('categories.txt')
label_encoder = get_label_encoder(categories)
X_train, y_train, X_test = get_train_test(train, test, label_encoder)
vocabulary = set(X_train).union(set(X_test))
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(vocabulary)
train_tokenized = tokenizer.texts_to_sequences(X_train)
test_tokenized = tokenizer.texts_to_sequences(X_test)
X_train = sequence.pad_sequences(train_tokenized, maxlen=hyper_params['max_sequence'])
X_test = sequence.pad_sequences(test_tokenized, maxlen=hyper_params['max_sequence'])

model = Sequential()
model.add(Embedding(len(vocabulary) + 1, hyper_params['embedding_size']))
model.add(LSTM(hyper_params['lstm_size']))
model.add(Dropout(1 - hyper_params['keep_probability']))
model.add(Dense(hyper_params['dense_size'], activation='relu'))
model.add(Dropout(1 - hyper_params['keep_probability']))
model.add(Dense(len(categories), activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=hyper_params['batch_size'], epochs=hyper_params['epochs'], 
                    validation_split=hyper_params['validation_split'], shuffle=True)

predictions = [label_encoder.inverse_transform([argmax(result)])[0] for result in model.predict(X_test)]
output_file = open('output2.txt', 'w')
for item in predictions:
    output_file.write("%s\n" % item)

Train on 18000 samples, validate on 2000 samples
Epoch 1/10
18000/18000 [==============================] - 102s 6ms/step - loss: 2.5478 - acc: 0.2429 - val_loss: 2.1357 - val_acc: 0.3645
Epoch 2/10
18000/18000 [==============================] - 94s 5ms/step - loss: 1.7220 - acc: 0.4943 - val_loss: 1.7513 - val_acc: 0.5040
Epoch 3/10
18000/18000 [==============================] - 95s 5ms/step - loss: 1.2172 - acc: 0.6469 - val_loss: 1.7883 - val_acc: 0.5015
Epoch 4/10
18000/18000 [==============================] - 97s 5ms/step - loss: 0.8858 - acc: 0.7452 - val_loss: 1.9359 - val_acc: 0.5090
Epoch 5/10
18000/18000 [==============================] - 93s 5ms/step - loss: 0.6626 - acc: 0.8081 - val_loss: 2.0290 - val_acc: 0.5245
Epoch 6/10
18000/18000 [==============================] - 90s 5ms/step - loss: 0.5022 - acc: 0.8596 - val_loss: 2.1995 - val_acc: 0.5250
Epoch 7/10
18000/18000 [==============================] - 109s 6ms/step - loss: 0.3951 - acc: 0.8892 - val_loss: 2.4529 - val_ac